In [86]:
from db import executeInput, execute,dropTable
import pandas as pd

# 175. Combine Two Tables

[LeetCode Link](https://leetcode.com/problems/combine-two-tables/) | **Easy** | **Topics:** Joins, Left Join

## Schema

**Table: `Person`**

| Column Name | Type    |
|-------------|---------|
| personId    | int     |
| lastName    | varchar |
| firstName   | varchar |

`personId` is the primary key (column with unique values) for this table.
This table contains information about the ID of some persons and their first and last names.

**Table: `Address`**

| Column Name | Type    |
|-------------|---------|
| addressId   | int     |
| personId    | int     |
| city        | varchar |
| state       | varchar |

`addressId` is the primary key (column with unique values) for this table.
Each row of this table contains information about the city and state of one person with ID = `PersonId`.

## Problem

Write a solution to report the first name, last name, city, and state of each person in the `Person` table. If the address of a `personId` is not present in the `Address` table, report `null` instead.

Return the result table in any order.


In [87]:
# SETUP
data = [[1, "Wang", "Allen"], [2, "Alice", "Bob"]]
person = pd.DataFrame(data, columns=["personId", "firstName", "lastName"]).astype(
    {"personId": "Int64", "firstName": "object", "lastName": "object"}
)
data = [[1, 2, "New York City", "New York"], [2, 3, "Leetcode", "California"]]
address = pd.DataFrame(data, columns=["addressId", "personId", "city", "state"]).astype(
    {"addressId": "Int64", "personId": "Int64", "city": "object", "state": "object"}
)

executeInput()

[]

In [88]:
#SQL Solution
query ="""
SELECT firstName, lastName, city, state
FROM Person p
LEFT JOIN Address a ON p.personId = a.personID
"""

pd.DataFrame(execute(query))

,0,1,2,3
0,Allen,Wang,None,None
1,Bob,Alice,New York City,New York


In [ ]:
#Cleanup
dropTable('person')
dropTable('address')

In [ ]:
#Pandas solution
import pandas as pd

def combine_two_tables(person: pd.DataFrame, address: pd.DataFrame) -> pd.DataFrame:
    merged = pd.merge(person, address, on="personId", how="left")
    return merged[["firstName", "lastName", "city", "state"]]
    


In [125]:
combine_two_tables(person, address)

,firstName,lastName,city,state
0,Wang,Allen,NaN,NaN
1,Alice,Bob,New York City,New York


# 181. Employees Earning More Than Their Managers

[LeetCode Link](https://leetcode.com/problems/employees-earning-more-than-their-managers/) | **Easy** | **Topics:** Database, Join

## Schema

**Table: `Employee`**

| Column Name | Type    |
|-------------|---------|
| id          | int     |
| name        | varchar |
| salary      | int     |
| managerId   | int     |

`id` is the primary key (column with unique values) for this table.
Each row of this table indicates the ID of an employee, their name, salary, and the ID of their manager.

## Problem

Write a solution to find the employees who earn more than their managers.

Return the result table in any order.

The result format is in the following example.

## Example 1

**Input:**

`Employee` table:

| id | name  | salary | managerId |
|----|-------|--------|-----------|
| 1  | Joe   | 70000  | 3         |
| 2  | Henry | 80000  | 4         |
| 3  | Sam   | 60000  | Null      |
| 4  | Max   | 90000  | Null      |

**Output:**

| Employee |
|----------|
| Joe      |

**Explanation:**
Joe is the only employee who earns more than his manager.


In [135]:
# SETUP
data = [
    [1, "Joe", 70000, 3],
    [2, "Henry", 80000, 4],
    [3, "Sam", 60000, None],
    [4, "Max", 90000, None],
]
employee = pd.DataFrame(data, columns=["id", "name", "salary", "managerId"]).astype(
    {"id": "Int64", "name": "object", "salary": "Int64", "managerId": "Int64"}
)

executeInput()

[]

In [136]:
#PostgreSQL Soltution
query="""
SELECT e1.name as Employee
FROM Employee e1
JOIN Employee e2 ON e1.managerid = e2.id
WHERE e1.salary > e2.salary
"""
pd.DataFrame(execute(query))

,0
0,Joe


In [137]:
#cleanup
dropTable('employee')

In [140]:
# Pandas Solution
import pandas as pd

def find_employees(employee: pd.DataFrame) -> pd.DataFrame:
    merged = pd.merge(employee, employee, left_on='managerId', right_on="id", suffixes=("", "_manager"))
    res = merged[merged["salary"] > merged["salary_manager"]][["name"]]
    res.columns = ['Employee']
    return res

In [141]:
find_employees(employee)

,Employee
0,Joe
